## Steps:
1. In the utils.py - Instantiate db with 
```python
    db_name="dvdrental" & include_tables = ["actor","film","film_actor"]
```
2. Initiliaze the LLM model.
3. Create a first chain, which will take query and return SQL query using database table schema.
4. Create a second chain which will take the SQL query as input from first chain, execute the query and provide the answer in natural language.

In [11]:
# langchain
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# local utils
from utils import db, get_schema, custom_str_parser, handle_error_query

In [8]:
# Langsmith - to debug LLM responses

import os
from dotenv import load_dotenv
from langsmith import Client

load_dotenv()

lcs = os.getenv("LANGCHAIN_SECRET")

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langchain-ollama-sql"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = lcs

client = Client()

## Load LLM and define chains

In [12]:
# Add the LLM downloaded from Ollama
ollama_llm = "mistral"
llm = ChatOllama(model=ollama_llm)

In [13]:
# ------------------------------------------------------------------------------------------------------------------------

# first chain
sql_chain_template = """Based on the table schema below, write a SQL query that would answer the user's question:
{schema}

Question: {question}
SQL Query:"""

sql_chain_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Given an input question, convert it to a SQL query. ",
        ),  # Brief context
        (
            "system",
            "Respond with only the SQL query, nothing else.",
        ),  # Emphasize desired output
        ("human", sql_chain_template),
    ]
)

sql_chain = (
    RunnablePassthrough.assign(schema=get_schema)
    | sql_chain_prompt
    # | llm.bind(stop=["\nSQLResult:"])
    | llm
    | StrOutputParser()
    | RunnableLambda(custom_str_parser)
)

# ------------------------------------------------------------------------------------------------------------------------

# second_chain
template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""  # noqa: E501

prompt_response = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Given an input question and SQL response, convert it to a natural "
            "language answer. No pre-amble.",
        ),
        ("human", template),
    ]
)

sql_run_chain = (
    RunnablePassthrough.assign(query=sql_chain)
    | RunnablePassthrough.assign(
        schema=get_schema, response=lambda x: handle_error_query(x)
    )
    | prompt_response
    | llm
    | StrOutputParser()
)

## Run Queries

#### Test if langchain can access the database

In [4]:
print(db.get_table_info())


CREATE TABLE actor (
	actor_id SERIAL NOT NULL, 
	first_name VARCHAR(45) NOT NULL, 
	last_name VARCHAR(45) NOT NULL, 
	last_update TIMESTAMP WITHOUT TIME ZONE DEFAULT now() NOT NULL, 
	CONSTRAINT actor_pkey PRIMARY KEY (actor_id)
)


CREATE TABLE film (
	film_id SERIAL NOT NULL, 
	title VARCHAR(255) NOT NULL, 
	description TEXT, 
	release_year INTEGER, 
	language_id SMALLINT NOT NULL, 
	rental_duration SMALLINT DEFAULT 3 NOT NULL, 
	rental_rate NUMERIC(4, 2) DEFAULT 4.99 NOT NULL, 
	length SMALLINT, 
	replacement_cost NUMERIC(5, 2) DEFAULT 19.99 NOT NULL, 
	rating mpaa_rating DEFAULT 'G'::mpaa_rating, 
	last_update TIMESTAMP WITHOUT TIME ZONE DEFAULT now() NOT NULL, 
	special_features TEXT[], 
	fulltext TSVECTOR NOT NULL, 
	CONSTRAINT film_pkey PRIMARY KEY (film_id), 
	CONSTRAINT film_language_id_fkey FOREIGN KEY(language_id) REFERENCES language (language_id) ON DELETE RESTRICT ON UPDATE CASCADE
)


CREATE TABLE film_actor (
	actor_id SMALLINT NOT NULL, 
	film_id SMALLINT NOT NULL, 
	

In [5]:
db.run("""SELECT * FROM actor LIMIT 5""")

"[(1, 'Penelope', 'Guiness', datetime.datetime(2013, 5, 26, 14, 47, 57, 620000)), (2, 'Nick', 'Wahlberg', datetime.datetime(2013, 5, 26, 14, 47, 57, 620000)), (3, 'Ed', 'Chase', datetime.datetime(2013, 5, 26, 14, 47, 57, 620000)), (4, 'Jennifer', 'Davis', datetime.datetime(2013, 5, 26, 14, 47, 57, 620000)), (5, 'Johnny', 'Lollobrigida', datetime.datetime(2013, 5, 26, 14, 47, 57, 620000))]"

In [18]:
db.run("""SELECT COUNT(*) FROM actor WHERE last_name = 'Penelope' AND first_name IS NOT NULL;""")

'[(0,)]'

#### We can run the first chain independently to test 1. We are getting valid SQL queries 2. The format of response is correct i.e. just the query

In [14]:
sql_chain.invoke({"question": "Give me count of actors"})

' SELECT COUNT(actor.actor_id) FROM actor;'

In [5]:
sql_chain.invoke({"question": "How many films has Penelope starred in?"})

" SELECT COUNT(DISTINCT film_id) FROM film_actor WHERE first_name = 'Penelope' AND last_name = (SELECT last_name FROM actor WHERE actor.actor_id = film_actor.actor_id);"

In [6]:
sql_chain.invoke({"question": "How many films has Penelope Guiness starred in?"})

" SELECT COUNT(*) FROM film_actor WHERE last_name = 'Guiness' AND first_name = 'Penelope';"

In [19]:
sql_chain.invoke(
    {"question": "Can you give me some movie names where Penelope Guiness starred?"}
)

" SELECT f.title FROM film f\nJOIN film_actor fa ON f.film_id = fa.film_id\nJOIN actor a ON fa.actor_id = a.actor_id\nWHERE a.first_name = 'Penelope' AND a.last_name = 'Guiness';"

In [16]:
sql_chain.invoke({"question": "Give me breakdown of films by release year"})

' SELECT release\\_year, COUNT(\\*) as film\\_count\nFROM film\nGROUP BY release\\_year;'

In [17]:
sql_chain.invoke({"question": "How many films were released in 2006?"})

' SELECT COUNT(*) FROM film WHERE release_year = 2006;'

In [18]:
sql_chain.invoke(
    {"question": "Going by the description, can you give me names of movies on robots?"}
)

" SELECT f.title \nFROM film f \nWHERE lower(f.description) LIKE '%robot%';"

In [19]:
sql_chain.invoke(
    {"question": "Give me names of movies where description contains Robot"}
)

" SELECT title FROM film WHERE description LIKE '%Robot%';"

### Invoking the main second chain on same queries as above.

In [15]:
print(sql_run_chain.invoke({"question": "Give me count of actors"}))

 The answer to your question is that there are 200 actors in the database.


In [16]:
print(sql_run_chain.invoke({"question": "How many films has Penelope starred in?"}))

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "last_name" does not exist
LINE 1:  SELECT COUNT(*) FROM film_actor WHERE last_name = 'Penelope...
                                               ^

[SQL:  SELECT COUNT(*) FROM film_actor WHERE last_name = 'Penelope' AND first_name IS NOT NULL;]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [9]:
print(
    sql_run_chain.invoke(
        {"question": "How many films has Penelope Guiness starred in?"}
    )
)

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "last_name" does not exist
LINE 1:  SELECT COUNT(*) FROM film_actor WHERE last_name = 'Guiness'...
                                               ^

[SQL:  SELECT COUNT(*) FROM film_actor WHERE last_name = 'Guiness' AND first_name = 'Penelope';]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [20]:
print(
    sql_run_chain.invoke(
        {"question": "Can you give me some movie names where Penelope Guiness starred?"}
    )
)

 The SQL response lists the movie titles where Penelope Guiness acted. Some of these movies are "Academy Dinosaur," "Anaconda Confessions," "Angels Life," "Bulworth Commandments," and so on. Other movies in the list include "Cheaper Clyde," "Color Philadelphia," "Elephant Trojan," and so forth.


In [21]:
print(sql_run_chain.invoke({"question": "Give me breakdown of films by release year"}))

 There are 1000 films that were released in the year 2006.


In [22]:
print(
    sql_run_chain.invoke(
        {
            "question": "Going by the description, can you give me names of movies on robots?"
        }
    )
)

 Based on the description in the `film` table, I can provide you with a list of movie titles that have the word "robots" mentioned in their descriptions. Here are some examples:

1. "RoboCop"
2. "The Terminator"
3. "Transformers"
4. "I, Robot"
5. "Metropolis" (This classic silent film also features robots, though the description may not specifically mention the word)

These are just a few examples that match the given SQL query. The actual number and specific titles may vary depending on the data in your database.


In [23]:
print(
    sql_run_chain.invoke(
        {"question": "Give me names of movies where description contains Robot"}
    )
)

 The SQL query returned the titles of movies in the film table where the description contains the word "Robot". Here are the names of those movies: Agent Truman, Alley Evolution, Clones Pinocchio, Control Anthem, Gunfighter Mussolini, Hollow Jeopardy, Holocaust Highball, Lost Bird, Mask Peach, Mine Titans, Rocketeer Mother, Rollercoaster Bringing, Runner Madigan, Secretary Rouge, Wizard Coldblooded.
